# Wrangling of financial data

## Imports

In [5]:
import os

import pandas as pd

## Constants

In [6]:
RAW_DATA_PATH = "../data/raw/datenabzug_sp500.xlsx"
PROCESSED_DATA_PATH = "../data/processed/"

# Sales

## Load and transforms quarterly sales data

In [120]:
def load_transform_sales_quarter(quarter):
    df = pd.read_excel(RAW_DATA_PATH, sheet_name=f"Sales Q{quarter}")
    df = df.T.reset_index()
    df.columns = df.iloc[0]
    df = df.drop(0)

    # parse company and variable name from Name column
    df["company"] = df["Name"].apply(
        lambda x: "".join(x.split(" - ")[:-2]).lstrip().rstrip()
        if len(x.split(" - ")) > 1  # to avoid exception for #ERROR entries
        else ""
    )
    df["variable"] = df["Name"].apply(
        lambda x: x.split(" - ")[-2].lstrip().rstrip()
        if len(x.split(" - ")) > 1  # to avoid exception for #ERROR entries
        else ""
    )

    # melt dataframe
    df = df.melt(id_vars=["company", "variable", "Name"])

    # extract year and quarter
    df["year"] = df[0].apply(lambda x: int(x.split(" ")[1]))
    df["quarter"] = df[0].apply(lambda x: int(x.split(" ")[0].replace("Q", "")))
    df = df.drop(["Name", 0], axis=1)
    df = df.dropna()

    df = df[df["company"] != ""]
    df = df[df["variable"] != ""]

    assert (
        len(df["variable"].unique()) == 1
    ), "Variable column should only contain 1 unique value!"

    df = df.rename(columns={"value": "interim_sales"})
    df = df.drop("variable", axis=1)
    df = df[["company", "year", "quarter", "interim_sales"]]
    return df


df_sales = pd.concat([load_transform_sales_quarter(q + 1) for q in range(4)])
print(
    f"Transformed interim sales dataframe contains {len(df_sales)} records "
    f"for {df_sales['company'].nunique()} companies from year {df_sales['year'].min()} to {df_sales['year'].max()}."
)
df_sales.head()

Transformed interim sales dataframe contains 34841 records for 500 companies from year 2002 to 2022.


,company,year,quarter,interim_sales
503,APPLE INC,2003,1,1475000.0
504,MICROSOFT CORP,2003,1,7835000.0
505,AMAZON.COM INC,2003,1,1083559.0
507,ALPHABET INC,2003,1,248618.0
508,UNITEDHEALTH GROUP,2003,1,6975000.0


## Save to CSV

In [121]:
sales_outname = "sales.csv"
df_sales.to_csv(os.path.join(PROCESSED_DATA_PATH, sales_outname))

# Balance sheet

## Load and transform balance sheet data

In [217]:
def load_transform_balance_sheet():

    df = pd.read_excel(RAW_DATA_PATH, sheet_name=f"Balance Sheet")
    df = df.T.reset_index()
    df.columns = df.iloc[0]
    df = df.drop(0)

    # add space to names of companies without space before the "-"
    # without this, we will later have difficulties in correctly parsing the variable names
    company_names_without_space = {
        "THERMO FISHER SCIENTIFIC",
        "ADOBE (NAS)",
        "CONSTELLATION BRANDS 'A'",
        "WALGREENS BOOTS ALLIANCE",
        "LYONDELLBASELL INDS.CL.A",
        "CITIZENS FINANCIAL GROUP",
        "MID-AMER.APT COMMUNITIES",
        "TERADYNE (XSC)",
        "UNITED AIRLINES HOLDINGS",
        "ALLIANT ENERGY (XSC)",
        "CBOE GLOBAL MARKETS(BTS)",
        "BIO-RAD LABORATORIES 'A'",
        "UNIVERSAL HEALTH SVS.'B'",
        "NEWELL BRANDS (XSC)",
    }

    def add_space_to_company_names(name, c_names=company_names_without_space):
        for c_name in c_names:
            name = name.replace(c_name, c_name + " ")
        return name

    df["Name"] = df["Name"].apply(add_space_to_company_names)

    # parse company, variable category and variable name from Name column
    df["company"] = df["Name"].apply(
        lambda x: x.split(" - ")[0] if len(x.split(" - ")) > 1 else ""
    )

    df["variable_category"] = df["Name"].apply(
        lambda x: x.split(" - ")[1] if len(x.split(" - ")) > 1 else ""
    )

    df["variable_name"] = df["Name"].apply(
        lambda x: x.split(" - ")[-1] if len(x.split(" - ")) > 1 else ""
    )

    # melt dataframe
    df = df.melt(id_vars=["company", "variable_category", "variable_name", "Name"])

    # rename 0 to year
    df = df.rename(columns={0: "year"})

    # remove records with error
    df = df[df["company"] != ""]

    # drop Name column
    df = df.drop("Name", axis=1)

    # drop variable category as only the variable names seem to be meaningful over different companies
    df = df.drop(["variable_category"], axis=1)
    df = df.rename(columns={"variable_name": "variable"})

    # pivot dataframe to create columns for variables
    df = df.pivot_table(
        index=["company", "year"], columns=["variable"], values="value"
    ).reset_index()

    return df


df_balance_sheet = load_transform_balance_sheet()

print(
    f"Transformed balance sheet dataframe contains {df_balance_sheet.shape[0]} records "
    f"with {df_balance_sheet.shape[1]} variables "
    f"for {df_balance_sheet['company'].nunique()} companies from year {df_balance_sheet['year'].min()} "
    f"to {df_balance_sheet['year'].max()}."
)

df_balance_sheet.head()

Transformed balance sheet dataframe contains 9893 records with 30 variables for 503 companies from year 2002 to 2022.


variable,company,year,ASSETS (TOTAL),BORROWINGS REPAYABLE < 1 YEAR,"CURRENT, DEPOSIT & OTHER A/CS",EQUITY CAP. AND RESERVES,NET CURRENT ASSETS,NET DEBT,ORDINARY SHARE CAPITAL,PREFERENCE CAPITAL,...,TOTAL DEFERRED & FUTURE TAX,TOTAL INSURANCE FUNDS,TOTAL INTANGIBLES,TOTAL INVESTMNTS (EX.ASSOC),TOTAL INVESTS -INSURANCE,TOTAL LOAN CAPITAL,TOTAL RESERVES,TOTAL STOCK AND W.I.P.,TRADE CREDITORS,TRADE DEBTORS
0,3M,2002,14454000.0,1373000.0,NaN,6086000.0,1787000.0,2277000.0,5000.0,0.0,...,317000.0,NaN,1213000.0,275000.0,NaN,1520000.0,6081000.0,2091000.0,753000.0,2786000.0
1,3M,2003,15119000.0,1237000.0,NaN,5993000.0,1602000.0,2761000.0,5000.0,0.0,...,136000.0,NaN,2167000.0,200000.0,NaN,2142000.0,5988000.0,1931000.0,1224000.0,2840000.0
2,3M,2004,17416990.0,1273000.0,NaN,7885000.0,2638000.0,1242000.0,9000.0,0.0,...,243000.0,NaN,2693000.0,197000.0,NaN,1805000.0,7876000.0,1816000.0,1413000.0,3162000.0
3,3M,2005,20576000.0,2166000.0,NaN,10378000.0,2649000.0,207000.0,9000.0,0.0,...,1152000.0,NaN,2932000.0,205000.0,NaN,798000.0,10369000.0,1897000.0,1818000.0,3311000.0
4,3M,2006,20402990.0,1072000.0,NaN,10100000.0,1877000.0,1368000.0,9000.0,0.0,...,1499000.0,NaN,3959000.0,7000.0,NaN,1368000.0,10091000.0,2162000.0,1711000.0,3121000.0


## Save to CSV

In [229]:
balance_sheet_outname = "balance_sheet.csv"
df_balance_sheet.to_csv(os.path.join(PROCESSED_DATA_PATH, balance_sheet_outname))

# Profit & loss

## Load and transform profit & loss data

In [250]:
def load_transform_profit_loss():
    df_sheets = []
    for i in range(2):
        df = pd.read_excel(RAW_DATA_PATH, sheet_name=f"Profit & Loss {i+1}")
        df = df.T.reset_index()
        df.columns = df.iloc[0]
        df = df.drop(0)
        df_sheets.append(df)

    df = pd.concat(df_sheets)

    # add space to names of companies without space before the "-"
    # without this, we will later have difficulties in correctly parsing the variable names
    company_names_without_space = {
        "THERMO FISHER SCIENTIFIC",
        "ADOBE (NAS)",
        "CONSTELLATION BRANDS 'A'",
        "WALGREENS BOOTS ALLIANCE",
        "LYONDELLBASELL INDS.CL.A",
        "CITIZENS FINANCIAL GROUP",
        "MID-AMER.APT COMMUNITIES",
        "TERADYNE (XSC)",
        "UNITED AIRLINES HOLDINGS",
        "ALLIANT ENERGY (XSC)",
        "CBOE GLOBAL MARKETS(BTS)",
        "BIO-RAD LABORATORIES 'A'",
        "UNIVERSAL HEALTH SVS.'B'",
        "NEWELL BRANDS (XSC)",
    }

    def add_space_to_company_names(name, c_names=company_names_without_space):
        for c_name in c_names:
            name = name.replace(c_name, c_name + " ")
        return name

    df["Name"] = df["Name"].apply(add_space_to_company_names)

    # parse company, variable category and variable name from Name column
    df["company"] = df["Name"].apply(
        lambda x: x.split(" - ")[0] if len(x.split(" - ")) > 1 else ""
    )

    df["variable_category"] = df["Name"].apply(
        lambda x: x.split(" - ")[1] if len(x.split(" - ")) > 1 else ""
    )

    df["variable_name"] = df["Name"].apply(
        lambda x: x.split(" - ")[-1] if len(x.split(" - ")) > 1 else ""
    )

    # melt dataframe
    df = df.melt(id_vars=["company", "variable_category", "variable_name", "Name"])

    # rename 0 to year
    df = df.rename(columns={0: "year"})

    # remove records with error
    df = df[df["company"] != ""]

    # drop Name column
    df = df.drop("Name", axis=1)

    # drop variable category as only the variable names seem to be meaningful over different companies
    df = df.drop(["variable_category"], axis=1)
    df = df.rename(columns={"variable_name": "variable"})
    
    # pivot dataframe to create columns for variables
    df = df.pivot_table(
        index=["company", "year"], columns=["variable"], values="value"
    ).reset_index()
    
    return df

df_profit_loss = load_transform_profit_loss()

print(
    f"Transformed profit & loss dataframe contains {df_profit_loss.shape[0]} records "
    f"with {df_profit_loss.shape[1]} variables "
    f"for {df_profit_loss['company'].nunique()} companies from year {df_profit_loss['year'].min()} "
    f"to {df_profit_loss['year'].max()}."
)

df_profit_loss.head()

Transformed profit & loss dataframe contains 9951 records with 42 variables for 503 companies from year 2002 to 2022.


variable,company,year,A.W.O. INTANGIBLES,AFTER TAX PROFIT-ADJ,CASH EARNINGS PER SHARE,COST OF SALES,DEPRECIATION,DIVIDENDS PER SHARE,EARNED FOR ORDINARY,EARNED FOR ORDINARY-ADJ,...,PROV. FOR BAD DEBTS,PUBLISHED AFTER TAX PROFIT,PUBLISHED CASH EPS,RESEARCH AND DEVT.,"SELLING, GEN.&ADMIN EXPENSES",TOTAL EMPLOYMENT COSTS,TOTAL INCOME,TOTAL INTEREST CHARGES,TOTAL SALES,TOTAL TAX CHARGE
0,3M,2002,173000.0,1484000.0,3.238,7388000.0,1089000.0,1.20,1430000.0,1430000.0,...,NaN,1484000.0,3.238,1064000.0,4825000.0,39000.0,16079000.0,124000.0,16079000.0,702000.0
1,3M,2003,100000.0,2039000.0,3.154,7421000.0,954000.0,1.24,1974000.0,1974000.0,...,NaN,2039000.0,3.154,1066000.0,4709000.0,NaN,16332000.0,80000.0,16332000.0,966000.0
2,3M,2004,41000.0,2455000.0,3.479,8321000.0,964000.0,1.32,2403000.0,2403000.0,...,NaN,2455000.0,3.479,1102000.0,5141000.0,NaN,18232000.0,84000.0,18232000.0,1202000.0
3,3M,2005,43000.0,3052000.0,4.659,8959000.0,999000.0,1.44,2990000.0,2990000.0,...,NaN,3052000.0,4.659,1143000.0,5475000.0,NaN,20010990.0,69000.0,20010990.0,1503000.0
4,3M,2006,48000.0,3289000.0,5.124,9395000.0,986000.0,1.68,3234000.0,3234000.0,...,NaN,3289000.0,5.124,1242000.0,5747000.0,NaN,21166990.0,82000.0,21166990.0,1694000.0


## Save to CSV

In [252]:
profit_loss_outname = "profit_loss.csv"
df_profit_loss.to_csv(os.path.join(PROCESSED_DATA_PATH, profit_loss_outname))